In [80]:
# prepares the notebook for "inline" graphing, and imports matplotlib library
%matplotlib inline
%config InlineBackend.figure_format='svg'
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from operator import attrgetter
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from os.path import isfile, join

In [81]:
# These functions are used for data analysis.

# these functions are used in determining the type
# of data we are looking at (posthesh, neghold etc)
def isIncreasing(L):
    return L[0] < L[len(L) - 1]
def isDecreasing(L):
    return L[0] > L[len(L) - 1]
def non_increasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))
def non_decreasing(L):
    return all(x<=y for x, y in zip(L, L[1:]))
def monotonic(L):
    return non_increasing(L) or non_decreasing(L)
def average(array):
    return sum(array) / len(array)

# this will return the type of data you give it
# it takes an array of voltages and currents as arguments
def sortIntoSide(voltage, current):
    if isIncreasing(voltage) and isIncreasing(current):
        return "posThresh"
    elif isDecreasing(voltage) and isDecreasing(current):
        return "posHold"
    elif isDecreasing(voltage) and isIncreasing(current):
        return "negThresh"
    elif isIncreasing(voltage) and isDecreasing(current):
        return "negHold"
    else: 
        print("???")

In [82]:
# this will take a file name and device size and
# get all of the meaningful stuff out of it
class DataFile:
    def __init__(self, filePath):
        self.filePath = filePath
        self.getFileNameFromPath()
        self.getDeviceTempFromName()
        self.getDeviceNumberFromName()
        self.getSampleNumberFromName()
        self.readAllData()

        # print(self.deviceTemp, self.deviceNumber, self.fileName)
        # self.graphAllData()

    def getFileNameFromPath(self):
        self.fileName = self.filePath.split("/data/")[1]

    def getDeviceTempFromName(self):
        self.deviceTemp = self.fileName.split("_")[5][:-1]

    def getDeviceNumberFromName(self):
        self.deviceNumber = self.fileName.split("_")[4]

    def getSampleNumberFromName(self):
        self.sampleNumber = self.fileName.split("_")[3]

    def readAllData(self):
        df5 = pd.read_csv(self.filePath, names=['Time (s)','Current (A)','Voltage (V)'],sep='\t',skiprows=1)
        data = df5.values
        df5.head()
        self.t = data[:,0]
        self.I = data[:,1]
        self.V = data[:,2]

    def graphAllData(self):
        plt.rcParams['figure.figsize'] = (5,3)
        plt.grid()
        plt.semilogy(self.V, self.I)
        plt.xlabel('Voltage (V)')
        plt.ylabel('Current (A)')
        plt.text(-1, 0.01, "Temp: " + self.deviceTemp + " K", fontsize=12)
        plt.text(-2, 0.01, "Device: " + self.deviceNumber, fontsize=12)
        plt.text(-3, 0.01, "Sample: " + self.sampleNumber, fontsize=12)
        plt.figure()
        

In [83]:
pathToData = os.getcwd() + "/data/" # the folder where all the data is stored
files = [] # stores all the files data as DataFile objects

# go into the data folder and extract all file names
for path in os.listdir(pathToData):
    if os.path.isfile(os.path.join(pathToData, path)):
        files.append(DataFile(pathToData + path))

print("total files: ", len(files))

filesToGraph = []

for file in files:
  if(len(file.deviceTemp) == 3):
    if(file.deviceNumber == "D18"):
      filesToGraph.append(file)

filesToGraph.sort(key=lambda x: x.deviceTemp)

for file in filesToGraph:
  file.graphAllData()


total files:  34
